### SP500 Stock Demo — Notebook 03: Train and Register Model

This notebook trains an XGBoost regressor to predict 3‑month ahead returns from hourly features, logs experiments, and registers the best model.

**What you will do**
- Build `TARGET_PCT_3M` using `lead()` per ticker over time
- Time-based split with safeguards to keep non-empty train/test
- Hyperparameter sweep with explicit experiment tracking (params + metrics)
- Select best model and evaluate train/test metrics
- Register in Snowflake Model Registry and enable explainability

**Outputs**
- Experiment: `SP500_XGB_RET3M`
- Registry model: `XGB_SP500_RET3M` (default set)
- Tables: `FEATURE_SHAP_GLOBAL_TOP`, optionally `FEATURE_IMPORTANCE_SP500`

**Structure**
1) Session + experiment setup
2) Create supervised target
3) Time-based split
4) Train + sweep + log metrics
5) Final selection + evaluation
6) Registry + explainability
7) Optional: list recent runs

**Key functions you will see**
- `lead(col('CLOSE'), horizon)` creates the future value for the target
- `Window.partition_by('TICKER').order_by(col('TS'))` defines per‑ticker ordering
- `Pipeline([...]).fit(train_df)` trains on‑snow; `model.predict(df)` scores on‑snow
- Metrics: `mean_squared_error`, `mean_absolute_percentage_error`, `r2_score`
- Registry: `Registry(...).log_model(...)` and `get_model(name).default = version`


In [ ]:
# 0) Imports and session/context
# - Attach to the active Snowflake session used by this notebook
# - Set warehouse, database, schema for all subsequent operations
# - Initialize Experiment Tracking for logging runs/metrics/params
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, lead, avg, sqrt
from snowflake.snowpark.functions import abs as sp_abs
from snowflake.snowpark.functions import pow as sp_pow
from snowflake.snowpark import Window
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.registry import Registry
from snowflake.ml.experiment.experiment_tracking import ExperimentTracking

session = get_active_session()
session.sql("USE WAREHOUSE DEMO_WH_M").collect()
session.sql("USE DATABASE SP500_STOCK_DEMO").collect()
session.sql("USE SCHEMA DATA").collect()

# Experiment Tracking: all runs in this notebook go under this experiment
exp = ExperimentTracking(session=session)
exp.set_experiment("SP500_XGB_RET3M")

# Quick sanity check the feature table is accessible
session.table('PRICE_FEATURES').limit(5).show()


### Step 1 — Session and experiment setup

In [ ]:
# 1) Build supervised dataset: create 3-month ahead target
# - Use Snowpark to define a per-ticker time window
# - Compute FUT_CLOSE as a 378-hour lead of CLOSE, then TARGET_PCT_3M = FUT_CLOSE/CLOSE - 1
# - Drop rows without a future value
hourly = session.table('PRICE_FEATURES')
horizon_hours = 378
win_order = Window.partition_by('TICKER').order_by(col('TS'))

ds = (
    hourly
    .with_column('FUT_CLOSE', lead(col('CLOSE'), horizon_hours).over(win_order))
    .with_column('TARGET_PCT_3M', (col('FUT_CLOSE')/col('CLOSE') - 1))
    .drop('FUT_CLOSE')
    .filter(col('TARGET_PCT_3M').is_not_null())
)

ds.limit(5).show()


### Step 2 — Create supervised target

In [ ]:
# 2) Time-based train/test split with safeguards
# - Start near the end of the dataset for test split; if empty due to lead horizon, back off in steps
# - Fallback: approximate an 80/20 boundary in time
from datetime import timedelta
from snowflake.snowpark.functions import max as sp_max

max_ts_ds = ds.select(sp_max(col('TS')).alias('mx')).collect()[0]['MX']
cutoff = None
train_df = None
test_df = None

for days in [30, 60, 90, 120]:
    try_cutoff = max_ts_ds - timedelta(days=days)
    train_try = ds.filter(col('TS') < try_cutoff)
    test_try = ds.filter(col('TS') >= try_cutoff)
    if train_try.count() > 0 and test_try.count() > 0:
        cutoff = try_cutoff
        train_df, test_df = train_try, test_try
        break

if cutoff is None:
    from snowflake.snowpark.functions import min as sp_min
    bounds = ds.select(sp_min(col('TS')).alias('mn'), sp_max(col('TS')).alias('mx')).collect()[0]
    mn_ts, mx_ts = bounds['MN'], bounds['MX']
    boundary = mn_ts + (mx_ts - mn_ts) * 0.8
    train_df = ds.filter(col('TS') < boundary)
    test_df = ds.filter(col('TS') >= boundary)
    cutoff = boundary

print({'cutoff': cutoff})
print('Train rows:', train_df.count())
print('Test rows:', test_df.count())


In [ ]:
# 3) Define feature/label/output columns + basic dataset stats (no training here)
feature_cols = ['RET_1','SMA_5','SMA_20','VOL_20','RSI_PROXY','VOLUME','CLOSE']
label_col = 'TARGET_PCT_3M'
output_col = 'PREDICTED_RETURN'

print({
    'total_rows': ds.count(),
    'train_rows': train_df.count(),
    'test_rows': test_df.count(),
    'num_features': len(feature_cols)
})


### Step 3 — Time-based split

In [ ]:
# 3b) Utilities: metrics wrappers and identifier sanitizer (no compute)
# - Compute RMSE/MAPE/R2 using Snowflake ML metrics on Snowpark DataFrames
# - Sanitize run names to valid Snowflake identifiers per docs
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import re


def compute_metrics_quick(df, y_col: str, yhat_col: str) -> dict:
    rmse = mean_squared_error(df=df, y_true_col_names=y_col, y_pred_col_names=yhat_col, squared=False)
    mape = mean_absolute_percentage_error(df=df, y_true_col_names=y_col, y_pred_col_names=yhat_col)
    r2 = r2_score(df=df, y_true_col_name=y_col, y_pred_col_name=yhat_col)
    return {'rmse': rmse, 'mape': mape, 'r2': r2}


def sanitize_identifier(name: str, prefix: str = "RUN_") -> str:
    cleaned = re.sub(r'[^A-Za-z0-9_$]', '_', name)
    if not re.match(r'^[A-Za-z_]', cleaned):
        cleaned = f"{prefix}{cleaned}"
    return cleaned[:255]



In [ ]:
# 3c) Precompute identifier-safe run names (no model training here)
# - Build param grid and derive sanitized run names
# - Display mapping for quick inspection before compute-heavy training
param_grid = []
for depth in [4, 6]:
    for lr in [0.1, 0.05]:
        for n in [100, 200]:
            display_name = f"xgb_depth={depth}_lr={lr}_n={n}"
            run_name = sanitize_identifier(display_name)
            param_grid.append({
                "depth": depth,
                "lr": lr,
                "n": n,
                "display_name": display_name,
                "run_name": run_name,
            })

# Quick check: show mapping of display_name -> run_name
try:
    from IPython.display import display
    display(pd.DataFrame(param_grid))
except Exception:
    print(param_grid)


In [ ]:
# 3d) Train/eval with precomputed names (logs each trial as a run)
# - Auto-suffix run names (_V2, _V3, ...) to avoid collisions on re-runs
# - Log params (incl. human-friendly display_name) and metrics per trial
rows = []
for cfg in param_grid:
    depth, lr, n = cfg["depth"], cfg["lr"], cfg["n"]
    display_name, base_name = cfg["display_name"], cfg["run_name"]

    suffix = 1
    while True:
        candidate = base_name if suffix == 1 else f"{base_name}_V{suffix}"
        try:
            with exp.start_run(candidate):
                exp.log_params({
                    "display_name": display_name,
                    "run_name_variant": candidate,
                    "model": "XGBRegressor",
                    "max_depth": int(depth),
                    "learning_rate": float(lr),
                    "n_estimators": int(n),
                    "subsample": 0.8,
                    "colsample_bytree": 0.8,
                    "feature_cols": feature_cols,
                    "label_col": label_col,
                    "output_col": output_col,
                    "time_split_cutoff": cutoff.isoformat() if cutoff else "auto",
                    "dataset_table": "PRICE_FEATURES",
                    "horizon_hours": int(horizon_hours),
                })

                xgb_tmp = XGBRegressor(
                    n_estimators=n,
                    max_depth=depth,
                    learning_rate=lr,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    input_cols=feature_cols,
                    label_cols=[label_col],
                    output_cols=[output_col],
                    random_state=42,
                )
                model_tmp = Pipeline(steps=[('xgb', xgb_tmp)]).fit(train_df)
                pred_tmp = model_tmp.predict(test_df).select(label_col, output_col)
                m = compute_metrics_quick(pred_tmp, label_col, output_col)
                rows.append({
                    'max_depth': depth,
                    'learning_rate': lr,
                    'n_estimators': n,
                    'rmse': m['rmse'],
                    'mape': m['mape'],
                    'r2': m['r2'],
                })

                exp.log_metrics({
                    "rmse": float(m["rmse"]),
                    "mape": float(m["mape"]),
                    "r2": float(m["r2"]),
                })
            break
        except Exception as e:
            msg = str(e)
            if "already exists" in msg and "Experiment" in msg:
                suffix += 1
                continue
            else:
                raise

hp_df = pd.DataFrame(rows)
hp_df = hp_df.dropna(subset=['mape', 'rmse']).sort_values('mape').reset_index(drop=True)


### Step 4 (cont.) — Consolidate sweep and train best model

- Build DataFrame of sweep results, drop nulls
- Train final model with best params by RMSE (fallback to baseline if empty)
- Plot compact tuning curve if results exist


In [ ]:
# 3) Define features and train XGBRegressor (+ compact hyperparam sweep with seaborn plot)
feature_cols = ['RET_1','SMA_5','SMA_20','VOL_20','RSI_PROXY','VOLUME','CLOSE']
label_col = 'TARGET_PCT_3M'
output_col = 'PREDICTED_RETURN'

# Print dataset sizes and feature count before training
print({
    'total_rows': ds.count(),
    'train_rows': train_df.count(),
    'test_rows': test_df.count(),
    'num_features': len(feature_cols)
})

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import re

# Notebook: use Snowflake ML metrics APIs directly (keyword-only args, singular names).
# Note: the server-side stored procedure path uses Snowpark-based metrics due to package availability.

def compute_metrics_quick(df, y_col: str, yhat_col: str) -> dict:
    rmse = mean_squared_error(df=df, y_true_col_names=y_col, y_pred_col_names=yhat_col, squared=False)
    mape = mean_absolute_percentage_error(df=df, y_true_col_names=y_col, y_pred_col_names=yhat_col)
    r2 = r2_score(df=df, y_true_col_name=y_col, y_pred_col_name=yhat_col)
    return {'rmse': rmse, 'mape': mape, 'r2': r2}


def sanitize_identifier(name: str, prefix: str = "RUN_") -> str:
    """Convert an arbitrary string to a valid Snowflake SQL identifier.
    - Replace invalid characters with underscores
    - Ensure it starts with a letter or underscore (else prefix)
    - Truncate to 255 characters
    See: https://docs.snowflake.com/en/sql-reference/identifiers-syntax
    """
    cleaned = re.sub(r'[^A-Za-z0-9_$]', '_', name)
    if not re.match(r'^[A-Za-z_]', cleaned):
        cleaned = f"{prefix}{cleaned}"
    return cleaned[:255]

# Compact grid similar to reference (learning_rate vs mape; hue=n_estimators; facet by max_depth)
max_depth_opts = [4, 6]
learning_rate_opts = [0.1, 0.05]
n_estimators_opts = [100, 200]

rows = []
for depth in max_depth_opts:
    for lr in learning_rate_opts:
        for n in n_estimators_opts:
            display_name = f"xgb_depth={depth}_lr={lr}_n={n}"
            run_name = sanitize_identifier(display_name)
            with exp.start_run(run_name):
                exp.log_params({
                    "display_name": display_name,
                    "model": "XGBRegressor",
                    "max_depth": int(depth),
                    "learning_rate": float(lr),
                    "n_estimators": int(n),
                    "subsample": 0.8,
                    "colsample_bytree": 0.8,
                    "feature_cols": feature_cols,
                    "label_col": label_col,
                    "output_col": output_col,
                    "time_split_cutoff": cutoff.isoformat() if cutoff else "auto",
                    "dataset_table": "PRICE_FEATURES",
                    "horizon_hours": int(horizon_hours),
                })

                xgb_tmp = XGBRegressor(
                    n_estimators=n,
                    max_depth=depth,
                    learning_rate=lr,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    input_cols=feature_cols,
                    label_cols=[label_col],
                    output_cols=[output_col],
                    random_state=42,
                )
                model_tmp = Pipeline(steps=[('xgb', xgb_tmp)]).fit(train_df)
                pred_tmp = model_tmp.predict(test_df).select(label_col, output_col)
                m = compute_metrics_quick(pred_tmp, label_col, output_col)
                rows.append({
                    'max_depth': depth,
                    'learning_rate': lr,
                    'n_estimators': n,
                    'rmse': m['rmse'],
                    'mape': m['mape'],
                    'r2': m['r2'],
                })

                exp.log_metrics({
                    "rmse": float(m["rmse"]),
                    "mape": float(m["mape"]),
                    "r2": float(m["r2"]),
                })

hp_df = pd.DataFrame(rows)
# Drop any null metric rows to avoid seaborn errors
hp_df = hp_df.dropna(subset=['mape', 'rmse']).sort_values('mape').reset_index(drop=True)

# Train final model with best params (by RMSE) if available
if not hp_df.empty:
    best = hp_df.sort_values('rmse').iloc[0].to_dict()
    xgb = XGBRegressor(
        n_estimators=int(best['n_estimators']),
        max_depth=int(best['max_depth']),
        learning_rate=float(best['learning_rate']),
        subsample=0.8,
        colsample_bytree=0.8,
        input_cols=feature_cols,
        label_cols=[label_col],
        output_cols=[output_col],
        random_state=42,
    )
    pipe = Pipeline(steps=[('xgb', xgb)])
    model = pipe.fit(train_df)
    print('Model trained with best config (by RMSE):', best)
else:
    # Fall back to baseline params if sweep failed
    best = {'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.05}
    xgb = XGBRegressor(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        input_cols=feature_cols,
        label_cols=[label_col],
        output_cols=[output_col],
        random_state=42,
    )
    pipe = Pipeline(steps=[('xgb', xgb)])
    model = pipe.fit(train_df)
    print('Sweep produced no metrics; trained baseline config:', best)

# Plot tuning curve similar to reference (learning_rate vs MAPE) if data available
if not hp_df.empty:
    sns.set_context('notebook', font_scale=0.9)
    # Use FacetGrid-like relplot to match style
    g = sns.relplot(
        data=hp_df,
        x='learning_rate', y='mape', hue='n_estimators', col='max_depth',
        kind='line', marker='o', height=4, aspect=1.2
    )
    g.set_titles('max_depth = {col_name}')
    g.set_axis_labels('learning_rate', 'MAPE')
else:
    print('Skipping tuning plot: no sweep results')

### Step 4 — Hyperparameter sweep + experiment logging

In [ ]:
# 4) Evaluate metrics (RMSE, MAPE, R2) using robust helper
train_pred = model.predict(train_df).select(label_col, output_col)
test_pred = model.predict(test_df).select(label_col, output_col)

train_metrics = compute_metrics_quick(train_pred, label_col, output_col)
test_metrics = compute_metrics_quick(test_pred, label_col, output_col)

print({'train': train_metrics, 'test': test_metrics})

# Log final evaluation and selection as a run for traceability
with exp.start_run("final_best_selection"):
    exp.log_params({
        "selected_by": "rmse",
        "best_max_depth": int(best["max_depth"]) if isinstance(best["max_depth"], (int, float, str)) else best["max_depth"],
        "best_learning_rate": float(best["learning_rate"]),
        "best_n_estimators": int(best["n_estimators"]),
        "feature_cols": feature_cols,
        "label_col": label_col,
        "output_col": output_col,
        "time_split_cutoff": cutoff.isoformat() if cutoff else "auto",
        "dataset_table": "PRICE_FEATURES",
        "horizon_hours": int(horizon_hours),
    })

    exp.log_metrics({
        "train_rmse": float(train_metrics["rmse"]),
        "train_mape": float(train_metrics["mape"]),
        "train_r2": float(train_metrics["r2"]),
        "test_rmse": float(test_metrics["rmse"]),
        "test_mape": float(test_metrics["mape"]),
        "test_r2": float(test_metrics["r2"]),
    })


### Step 5 — Final selection and evaluation

- Select best configuration by RMSE (fallback to baseline if needed) and refit
- Compute train/test metrics (RMSE, MAPE, R2)
- Log `final_best_selection` run with selected params and metrics

Tip: For identifier safety, run names are sanitized (letters/digits/`_`/`$`, start with letter/underscore, max 255 chars).

In [ ]:
# 5) Register in Model Registry (enable Snowflake explainability)
reg = Registry(session=session, database_name='SP500_STOCK_DEMO', schema_name='DATA')
model_name = 'XGB_SP500_RET3M'

models_df = reg.show_models()
if models_df.empty or model_name not in models_df['name'].to_list():
    version = 'V_1'
else:
    import ast, builtins
    max_v = builtins.max([int(v.split('_')[-1]) for v in ast.literal_eval(models_df.loc[models_df['name']==model_name,'versions'].values[0])])
    version = f'V_{max_v+1}'

# Provide background data to enable SHAP explainability in the Model Registry
bg_pd = train_df.select(feature_cols).limit(1000).to_pandas()

mv = reg.log_model(
    model,
    model_name=model_name,
    version_name=version,
    conda_dependencies=['snowflake-ml-python'],
    comment='XGBRegressor predicting 3-month forward return from hourly features',
    metrics={
        'train_r2': train_metrics['r2'],
        'test_r2': test_metrics['r2'],
        'train_rmse': train_metrics['rmse'],
        'test_rmse': test_metrics['rmse'],
        'train_mape': train_metrics['mape'],
        'test_mape': test_metrics['mape'],
    },
    sample_input_data=bg_pd,
    options={'relax_version': False, 'enable_explainability': True}
)
reg.get_model(model_name).default = version

# Log registry pointers into the final run context
with exp.start_run("final_best_selection_regref"):
    exp.log_params({
        "registry_model_name": model_name,
        "registry_version": version,
    })

print({'model_name': model_name, 'version': version})


### Step 6 — Model Registry and experiment linkage

- Register model and set default version in Snowflake Model Registry
- Enable explainability by providing background data
- Record `registry_model_name` and `registry_version` in a short run to link experiments to the registry


In [ ]:
# 6) Snowflake explainability: compute SHAP-based global feature importance + seaborn bar plot
# explain() availability was enabled during logging (cell 5) using background data.
from snowflake.snowpark.functions import abs as sp_abs, col as sp_col
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

reg = Registry(session=session, database_name='SP500_STOCK_DEMO', schema_name='DATA')
mv_explain = reg.get_model('XGB_SP500_RET3M').default

# Take a small sample of inputs (features only) for explanations
sample_sp = ds.select(*feature_cols).limit(200)

# Run SHAP explain through Model Registry (Snowflake-built explainability)
explanations_sp = mv_explain.run(sample_sp, function_name='explain')

# Aggregate mean absolute SHAP per feature
ex_pd = explanations_sp.to_pandas()
cols = list(ex_pd.columns)
shap_map = {}
for f in feature_cols:
    if f in cols:
        shap_map[f] = f
    elif f"SHAP_{f}" in cols:
        shap_map[f] = f"SHAP_{f}"

imp_rows = []
for f, c in shap_map.items():
    imp = float(pd.Series(ex_pd[c]).abs().mean())
    imp_rows.append({'feature': f, 'mean_abs_shap': imp})
imp_df = pd.DataFrame(imp_rows).sort_values('mean_abs_shap', ascending=False)

# Persist and plot
session.create_dataframe(imp_df).write.save_as_table('FEATURE_SHAP_GLOBAL_TOP', mode='overwrite')

sns.set_context('notebook', font_scale=0.9)
plt.figure(figsize=(8, 4))
sns.barplot(data=imp_df, x='mean_abs_shap', y='feature', orient='h')
plt.title('Global feature importance (mean |SHAP|)')
plt.xlabel('Mean |SHAP|')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()
print(imp_df.head(10))


In [ ]:
# 6b) Explainability: feature importances (from sklearn view) — optional quick view
sk_pipe = model.to_sklearn()
# Find xgb step
xgb_step = None
for name, step in sk_pipe.named_steps.items():
    if 'xgb' in name.lower():
        xgb_step = step
        break

if xgb_step is not None and hasattr(xgb_step, 'feature_importances_'):
    import pandas as pd
    fi = pd.DataFrame({
        'feature': feature_cols,
        'importance': xgb_step.feature_importances_.tolist()[:len(feature_cols)]
    }).sort_values('importance', ascending=False)
    fi_sp = session.create_dataframe(fi)
    fi_sp.write.save_as_table('FEATURE_IMPORTANCE_SP500', mode='overwrite')
    print('Saved FEATURE_IMPORTANCE_SP500')
else:
    print('Feature importances not available from sklearn object.')


In [ ]:
# 7) Optional: List recent runs from Experiment Tracking (for live demo)
print("Experiment:", "SP500_XGB_RET3M")

runs = None
used_method = None

candidates = [
    ("show_runs(limit=20)", lambda: exp.show_runs(limit=20)),
    ("list_runs(limit=20)", lambda: exp.list_runs(limit=20)),
    ("search_runs(limit=20)", lambda: exp.search_runs(limit=20)),
    ("get_runs(limit=20)", lambda: exp.get_runs(limit=20)),
    ("show_runs()", lambda: exp.show_runs()),
    ("list_runs()", lambda: exp.list_runs()),
    ("search_runs()", lambda: exp.search_runs()),
    ("get_runs()", lambda: exp.get_runs()),
]

for label, fn in candidates:
    try:
        runs = fn()
        used_method = label
        break
    except Exception:
        pass

if runs is not None:
    print(f"Listed runs via: {used_method}")
    try:
        import pandas as pd
        from IPython.display import display
        display(runs)
    except Exception:
        print(runs)
else:
    print("Could not list runs programmatically. Open Snowsight > ML > Experiments and select 'SP500_XGB_RET3M'.")
